In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#Data Loading
train_data = pd.read_csv("../input/housing-prices-competition-for-kaggle-learn-users/train.csv",index_col = 0)
test_data = pd.read_csv("../input/housing-prices-competition-for-kaggle-learn-users/test.csv",index_col = 0)

In [ ]:
#Label Selection
train_col = train_data
test_col = test_data.columns.tolist()


features = train_data.copy()
lab = features.pop("SalePrice")




In [ ]:
features.shape




In [ ]:
#Replace Encoding
tokens ={'Ex':5 ,'Gd':4 ,'TA':3 ,'Fa':2 ,'Po':1, 'NA':0}    
replace_list = [col for col in features if features[col].value_counts().index.to_numpy()[0] in tokens.keys()]
replace = features[replace_list].fillna('NA').replace(regex=tokens)




replace.corrwith(lab).abs().sort_values(ascending=False)

In [ ]:
features = features.drop(replace_list, axis=1)

#Splitting Categorical/Numerical Features
cat = features.select_dtypes(include='object')
cat = cat.fillna('NA')

num = features.select_dtypes(exclude='object')
#Filling Missing Values in Num
num = num.fillna(0)

#Ordinal/OneHot Encoding Split
ordinal_list = [col for col in cat if cat[col].nunique() ==2]

##############################################################




 

oneHot = cat.drop(ordinal_list,axis=1)
oneHot = pd.get_dummies(oneHot)


oneHot.corrwith(lab).abs().sort_values(ascending=False).head(17)


In [ ]:
ordinal = OrdinalEncoder().fit_transform(cat)
ordinal = pd.DataFrame(ordinal, index=cat.index, columns=cat.columns)
ordinal.corrwith(lab).abs().sort_values(ascending=False)

In [ ]:

#All Numeric Data  #replace   #oneHot  #ordinal 
feat = pd.concat([num,replace,oneHot,ordinal], axis=1)

feat.shape

In [ ]:
feat_index = feat.corrwith(lab).abs().sort_values(ascending=False).head(45).index
feat.corrwith(lab).abs().sort_values(ascending=False).head(44)





In [ ]:
feat = feat[feat_index]
#Train/Validation Data Split
from sklearn.model_selection import train_test_split

train_feat, test_feat, train_lab, test_lab = train_test_split(feat, lab, test_size=0.1, random_state=0)



In [ ]:
#Training the Model
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

model = XGBRegressor(n_estimators = 1500, learning_rate = 0.005)
model.fit(train_feat,train_lab, eval_set=[(test_feat,test_lab)], early_stopping_rounds=5, verbose = 0)




In [ ]:
#Evaluation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#reg = LinearRegression().fit(train_feat,train_lab)


pred_test_lab = model.predict(test_feat)




print('mean_squared_error',mean_squared_error(test_lab,pred_test_lab ) )
print('mean_absolute_error',mean_absolute_error(test_lab,pred_test_lab ) )
print('root mean_squared_error',np.sqrt(mean_squared_error(test_lab,pred_test_lab )) )
print('No of features',train_feat.shape )
print(train_feat.columns.tolist())



"""


mean_squared_error 691,435,179.8383178
mean_absolute_error 16984.128157106163      *********************____54%____********************
root mean_squared_error 26295.155063971724          
No of features (1022, 25)


mean_squared_error 690928850.1944076
mean_absolute_error 16984.05040667808
root mean_squared_error 26285.5254882684
No of features (1022, 25)


mean_squared_error 693699851.8705637
mean_absolute_error 17015.31421232877
root mean_squared_error 26338.18239496727
No of features (1022, 23)
['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 
'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 
'YearRemodAdd', 'MasVnrArea', 'Fireplaces', 'BsmtFinSF1', 'WoodDeckSF', 
'2ndFlrSF', 'OpenPorchSF', 'HalfBath', 'LotArea', 'GarageYrBlt', 'BsmtFullBath',
'BsmtUnfSF', 'LotFrontage', 'BedroomAbvGr']



mean_squared_error 571644094.8095632
mean_absolute_error 14936.241598886987
root mean_squared_error 23909.079756643983
No of features (1314, 45)
['OverallQual', 'GrLivArea', 'ExterQual', 'KitchenQual', 'GarageCars', 
'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'BsmtQual', 'FullBath', 'TotRmsAbvGrd',
'YearBuilt', 'FireplaceQu', 'YearRemodAdd', 'Foundation_PConc', 'MasVnrArea', 
'Fireplaces', 'BsmtFinType1_GLQ', 'HeatingQC', 'GarageFinish', 'GarageFinish_Fin',
'GarageType', 'GarageFinish_Unf', 'Neighborhood_NridgHt', 'BsmtFinSF1', 'Foundation', 
'MasVnrType_None', 'SaleType_New', 'GarageType_Detchd', 'SaleCondition_Partial',
'Foundation_CBlock', 'GarageType_Attchd', 'MasVnrType_Stone', 'Neighborhood_NoRidge',
'WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'BsmtExposure_Gd', 'Exterior2nd_VinylSd', 
'Exterior1st_VinylSd', 'BsmtExposure', 'MSZoning_RM', 'HalfBath', 'GarageQual', 'LotShape_Reg']


"""

In [ ]:
###replace Enconding
replace_test = test_data[replace_list].fillna('NA').replace(regex=tokens)
test_data = test_data.drop(replace_list, axis=1)

#Splitting Categorical/Numerical Test Features
cat_test = test_data.select_dtypes(include='object')
cat_test = cat_test.fillna('NA')

num_test = test_data.select_dtypes(exclude='object')
#Filling Missing Values in Num
num_test = num_test.fillna(0)

#OneHot Enconding
oneHot_test = cat_test.drop(ordinal_list,axis=1)
oneHot_test = pd.get_dummies(oneHot_test)

#Ordinal Encoding
ordinal_test = OrdinalEncoder().fit_transform(cat_test)
ordinal_test = pd.DataFrame(ordinal_test, index=cat_test.index, columns=cat_test.columns)


test = pd.concat([num_test,replace_test,oneHot_test,ordinal_test], axis=1)

test = test[feat_index]

In [ ]:


prd_data = model.predict(test)

submission = pd.DataFrame(prd_data, index=test_data.index, columns =['SalePrice'])

submission.to_csv('SalePrice_corr45_14936.csv')

submission.head()

In [ ]:
test_lab.head()

pred_test_lab_df = pd.DataFrame(pred_test_lab, index = test_feat.index, columns = ['Prediction'])
error = test_lab - pred_test_lab_df.Prediction

pd.concat([test_lab,pred_test_lab_df, error], axis=1)

